In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!pip install openpyxl

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **1990~2021년 데이터 로드**

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/6조 (final)/data/new/gtd_199902021_summary.xlsx")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173597 entries, 0 to 173596
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       173597 non-null  int64  
 1   eventid          173597 non-null  int64  
 2   iyear            173597 non-null  int64  
 3   imonth           173597 non-null  int64  
 4   iday             173597 non-null  int64  
 5   country_txt      173597 non-null  object 
 6   region_txt       173597 non-null  object 
 7   provstate        173597 non-null  object 
 8   latitude         171066 non-null  float64
 9   longitude        171065 non-null  float64
 10  city             173170 non-null  object 
 11  success          173597 non-null  int64  
 12  summary          147370 non-null  object 
 13  attacktype1_txt  173597 non-null  object 
 14  targtype1_txt    173597 non-null  object 
 15  weaptype1_txt    173597 non-null  object 
 16  nkill            166327 non-null  floa

## **nkill, nwound 결측치 파악**

In [ ]:
columns_to_extract = [
    'eventid',
    'iyear', 'imonth', 'iday',
    'success',
    'country_txt', 'region_txt', 'provstate','city',
    'latitude', 'longitude',
    'attacktype1_txt', 'targtype1_txt', 'natlty1_txt', 'weaptype1_txt',
    'nkill', 'nwound',
    'gname'
]

In [ ]:
# 결측치 건수 계산
null_counts = df[columns_to_extract].isnull().sum()

# 결측치 비율 계산 (백분율로)
null_ratios = (df[columns_to_extract].isnull().mean() * 100).round(2)

# 결과 데이터프레임 생성
null_summary = pd.DataFrame({
    'Null Count': null_counts,
    'Null Ratio (%)': null_ratios
})

null_summary


,Null Count,Null Ratio (%)
eventid,0,0.00
iyear,0,0.00
imonth,0,0.00
iday,0,0.00
success,0,0.00
country_txt,0,0.00
region_txt,0,0.00
provstate,0,0.00
city,427,0.25
latitude,2531,1.46


### **success + attacktype 을 기준으로 평균 VS 중앙값 왜도를 비교**

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew

# 복사본 생성
df_mean_fill = df.copy()
df_median_fill = df.copy()

# 평균으로 채우기 (success + attacktype1_txt 조합 기준)
df_mean_fill['nkill'] = df_mean_fill['nkill'].fillna(
    df_mean_fill.groupby(['success', 'attacktype1_txt'])['nkill'].transform('mean')
)
df_mean_fill['nwound'] = df_mean_fill['nwound'].fillna(
    df_mean_fill.groupby(['success', 'attacktype1_txt'])['nwound'].transform('mean')
)

# 중앙값으로 채우기 (success + attacktype1_txt 조합 기준)
df_median_fill['nkill'] = df_median_fill['nkill'].fillna(
    df_median_fill.groupby(['success', 'attacktype1_txt'])['nkill'].transform('median')
)
df_median_fill['nwound'] = df_median_fill['nwound'].fillna(
    df_median_fill.groupby(['success', 'attacktype1_txt'])['nwound'].transform('median')
)

# 요약표 생성: 평균 / 중앙값 / 왜도 비교
summary = pd.DataFrame({
    '기준': ['nkill (원본)', 'nkill (평균)', 'nkill (중앙값)',
           'nwound (원본)', 'nwound (평균)', 'nwound (중앙값)'],
    '평균': [
        np.round(df['nkill'].mean(), 2),
        np.round(df_mean_fill['nkill'].mean(), 2),
        np.round(df_median_fill['nkill'].mean(), 2),
        np.round(df['nwound'].mean(), 2),
        np.round(df_mean_fill['nwound'].mean(), 2),
        np.round(df_median_fill['nwound'].mean(), 2),
    ],
    '중앙값': [
        np.round(df['nkill'].median(), 2),
        np.round(df_mean_fill['nkill'].median(), 2),
        np.round(df_median_fill['nkill'].median(), 2),
        np.round(df['nwound'].median(), 2),
        np.round(df_mean_fill['nwound'].median(), 2),
        np.round(df_median_fill['nwound'].median(), 2),
    ],
    '왜도(Skewness)': [
        np.round(skew(df['nkill'].dropna()), 2),
        np.round(skew(df_mean_fill['nkill']), 2),
        np.round(skew(df_median_fill['nkill']), 2),
        np.round(skew(df['nwound'].dropna()), 2),
        np.round(skew(df_mean_fill['nwound']), 2),
        np.round(skew(df_median_fill['nwound']), 2),
    ]
})

summary


,기준,평균,중앙값,왜도(Skewness)
0,nkill (원본),2.50,1.0,58.75
1,nkill (평균),2.53,1.0,59.95
2,nkill (중앙값),2.41,1.0,59.91
3,nwound (원본),3.35,0.0,202.60
4,nwound (평균),3.31,0.0,210.58
5,nwound (중앙값),3.12,0.0,210.68


### **0 구분하는 기준 추가**
- 0의 비율을 보고 0의 데이터와 0이 아닌 데이터를 나눈 후 success + attacktype 적용
- 총 3가지의 기준으로 나누어 본다.

In [ ]:
# nkill
nkill_0 = (df['nkill'] == 0).sum()
nkill_nonzero = (df['nkill'] > 0).sum()

# nwound
nwound_0 = (df['nwound'] == 0).sum()
nwound_nonzero = (df['nwound'] > 0).sum()


In [ ]:
print(f"nkill == 0: {nkill_0}건")
print(f"nkill > 0: {nkill_nonzero}건")

print(f"nwound == 0: {nwound_0}건")
print(f"nwound > 0: {nwound_nonzero}건")


nkill == 0: 80402건
nkill > 0: 85925건
nwound == 0: 92959건
nwound > 0: 67428건


In [ ]:
nkill_total = df['nkill'].notnull().sum()
print("nkill 분포")
print(f"nkill == 0   : {nkill_0}건 ({nkill_0 / nkill_total:.2%})")
print(f"nkill > 0    : {nkill_nonzero}건 ({nkill_nonzero / nkill_total:.2%})")

nkill 분포
nkill == 0   : 80402건 (48.34%)
nkill > 0    : 85925건 (51.66%)


In [ ]:
nwound_total = df['nwound'].notnull().sum()
print("nwound 분포")
print(f"nwound == 0  : {nwound_0}건 ({nwound_0 / nwound_total:.2%})")
print(f"nwound > 0   : {nwound_nonzero}건 ({nwound_nonzero / nwound_total:.2%})")

nwound 분포
nwound == 0  : 92959건 (57.96%)
nwound > 0   : 67428건 (42.04%)


### **해석!**
- 사망자, 부상자 : 0이 차지하는 비율이 비슷하다.
- 따로 보는 것이 좋겠다 판단.

### **0, >0 그룹으로 나누어 본다.**

In [ ]:
from scipy.stats import skew

# 0 vs >0 그룹 생성
def assign_groups(df):
    df['nkill_group'] = np.where(df['nkill'] == 0, '0', '>0')
    df['nwound_group'] = np.where(df['nwound'] == 0, '0', '>0')
    return df

df = assign_groups(df)
df_mean_fill = assign_groups(df_mean_fill)
df_median_fill = assign_groups(df_median_fill)

# 그룹 요약 함수 (평균, 중앙값, 왜도 포함)
def grouped_summary_with_skew(df, var, group_col):
    result = df.groupby(['success', 'attacktype1_txt', group_col])[var].agg(
        평균='mean',
        중앙값='median',
        왜도=lambda x: skew(x.dropna()) if x.dropna().nunique() > 1 else np.nan
    ).reset_index()
    return result

# nkill 요약
nkill_orig = grouped_summary_with_skew(df, 'nkill', 'nkill_group')
nkill_mean = grouped_summary_with_skew(df_mean_fill, 'nkill', 'nkill_group')
nkill_median = grouped_summary_with_skew(df_median_fill, 'nkill', 'nkill_group')

# nwound 요약
nwound_orig = grouped_summary_with_skew(df, 'nwound', 'nwound_group')
nwound_mean = grouped_summary_with_skew(df_mean_fill, 'nwound', 'nwound_group')
nwound_median = grouped_summary_with_skew(df_median_fill, 'nwound', 'nwound_group')


In [ ]:
print("📌 nkill (원본)")
display(nkill_orig)

print("📌 nkill (평균 대체)")
display(nkill_mean)

print("📌 nkill (중앙값 대체)")
display(nkill_median)

print("📌 nwound (원본)")
display(nwound_orig)

print("📌 nwound (평균 대체)")
display(nwound_mean)

print("📌 nwound (중앙값 대체)")
display(nwound_median)


📌 nkill (원본)


,success,attacktype1_txt,nkill_group,평균,중앙값,왜도
0,0,Armed Assault,0,0.000000,0.0,NaN
1,0,Armed Assault,>0,4.561111,2.0,3.838476
2,0,Assassination,0,0.000000,0.0,NaN
3,0,Assassination,>0,2.855385,2.0,4.247923
4,0,Bombing/Explosion,0,0.000000,0.0,NaN
5,0,Bombing/Explosion,>0,4.366448,2.0,6.018815
6,0,Facility/Infrastructure Attack,0,0.000000,0.0,NaN
7,0,Facility/Infrastructure Attack,>0,4.937500,2.5,2.742176
8,0,Hijacking,0,0.000000,0.0,NaN
9,0,Hijacking,>0,2.750000,2.0,1.550540


📌 nkill (평균 대체)


,success,attacktype1_txt,nkill_group,평균,중앙값,왜도
0,0,Armed Assault,0,0.000000,0.000000,NaN
1,0,Armed Assault,>0,4.094587,2.000000,4.129865
2,0,Assassination,0,0.000000,0.000000,NaN
3,0,Assassination,>0,2.764991,2.000000,4.307895
4,0,Bombing/Explosion,0,0.000000,0.000000,NaN
5,0,Bombing/Explosion,>0,4.013473,2.000000,6.236476
6,0,Facility/Infrastructure Attack,0,0.000000,0.000000,NaN
7,0,Facility/Infrastructure Attack,>0,4.184953,2.000000,2.976006
8,0,Hijacking,0,0.000000,0.000000,NaN
9,0,Hijacking,>0,2.576923,2.000000,1.614906


📌 nkill (중앙값 대체)


...

,success,attacktype1_txt,nwound_group,평균,중앙값,왜도
0,0,Armed Assault,0,0.000000,0.0,NaN
1,0,Armed Assault,>0,4.484009,2.0,3.825566
2,0,Assassination,0,0.000000,0.0,NaN
3,0,Assassination,>0,2.948238,1.0,8.686328
4,0,Bombing/Explosion,0,0.000000,0.0,NaN
5,0,Bombing/Explosion,>0,6.221184,2.0,6.067350
6,0,Facility/Infrastructure Attack,0,0.000000,0.0,NaN
7,0,Facility/Infrastructure Attack,>0,1.384615,1.0,1.387577
8,0,Hijacking,0,0.000000,0.0,NaN
9,0,Hijacking,>0,3.750000,2.0,2.194625


📌 nwound (평균 대체)


,success,attacktype1_txt,nwound_group,평균,중앙값,왜도
0,0,Armed Assault,0,0.000000,0.000000,NaN
1,0,Armed Assault,>0,3.173358,1.000000,4.625100
2,0,Assassination,0,0.000000,0.000000,NaN
3,0,Assassination,>0,2.906021,1.000000,8.853012
4,0,Bombing/Explosion,0,0.000000,0.000000,NaN
5,0,Bombing/Explosion,>0,4.608256,1.000000,6.995085
6,0,Facility/Infrastructure Attack,0,0.000000,0.000000,NaN
7,0,Facility/Infrastructure Attack,>0,0.739115,1.000000,1.029483
8,0,Hijacking,0,0.000000,0.000000,NaN
9,0,Hijacking,>0,3.513986,2.000000,2.318001


📌 nwound (중앙값 대체)


,success,attacktype1_txt,nwound_group,평균,중앙값,왜도
0,0,Armed Assault,0,0.000000,0.0,NaN
1,0,Armed Assault,>0,4.484009,2.0,3.825566
2,0,Assassination,0,0.000000,0.0,NaN
3,0,Assassination,>0,2.876591,1.0,8.830123
4,0,Bombing/Explosion,0,0.000000,0.0,NaN
5,0,Bombing/Explosion,>0,6.221184,2.0,6.067350
6,0,Facility/Infrastructure Attack,0,0.000000,0.0,NaN
7,0,Facility/Infrastructure Attack,>0,1.384615,1.0,1.387577
8,0,Hijacking,0,0.000000,0.0,NaN
9,0,Hijacking,>0,3.750000,2.0,2.194625


In [ ]:
# 복사: 원본, 평균 대체, 중앙값 대체에 "대체방법" 열 추가
nkill_orig['대체방법'] = '원본'
nkill_mean['대체방법'] = '평균 대체'
nkill_median['대체방법'] = '중앙값 대체'

# 세 개를 하나로 세로 병합
nkill_combined = pd.concat([nkill_orig, nkill_mean, nkill_median], ignore_index=True)

# 열 순서 정리
nkill_combined = nkill_combined[['대체방법', 'success', 'attacktype1_txt', 'nkill_group', '평균', '중앙값', '왜도']]

In [ ]:
for attack_type in nkill_combined['attacktype1_txt'].unique():
  print(f"\n📌 공격 유형: {attack_type}")
  display(nkill_combined[nkill_combined['attacktype1_txt'] == attack_type])


📌 공격 유형: Armed Assault


,대체방법,success,attacktype1_txt,nkill_group,평균,중앙값,왜도
0,원본,0,Armed Assault,0,0.000000,0.0,NaN
1,원본,0,Armed Assault,>0,4.561111,2.0,3.838476
18,원본,1,Armed Assault,0,0.000000,0.0,NaN
19,원본,1,Armed Assault,>0,4.940695,2.0,31.928821
36,평균 대체,0,Armed Assault,0,0.000000,0.0,NaN
37,평균 대체,0,Armed Assault,>0,4.094587,2.0,4.129865
54,평균 대체,1,Armed Assault,0,0.000000,0.0,NaN
55,평균 대체,1,Armed Assault,>0,4.900275,2.0,32.479091
72,중앙값 대체,0,Armed Assault,0,0.000000,0.0,NaN
73,중앙값 대체,0,Armed Assault,>0,4.561111,2.0,3.838476



📌 공격 유형: Assassination


,대체방법,success,attacktype1_txt,nkill_group,평균,중앙값,왜도
2,원본,0,Assassination,0,0.000000,0.0,NaN
3,원본,0,Assassination,>0,2.855385,2.0,4.247923
20,원본,1,Assassination,0,0.000000,0.0,NaN
21,원본,1,Assassination,>0,1.627900,1.0,8.902592
38,평균 대체,0,Assassination,0,0.000000,0.0,NaN
39,평균 대체,0,Assassination,>0,2.764991,2.0,4.307895
56,평균 대체,1,Assassination,0,0.000000,0.0,NaN
57,평균 대체,1,Assassination,>0,1.627846,1.0,8.910203
74,중앙값 대체,0,Assassination,0,0.000000,0.0,NaN
75,중앙값 대체,0,Assassination,>0,2.855385,2.0,4.247923



📌 공격 유형: Bombing/Explosion


...

,대체방법,success,attacktype1_txt,nkill_group,평균,중앙값,왜도
12,원본,0,Hostage Taking (Kidnapping),0,0.000000,0.000000,NaN
13,원본,0,Hostage Taking (Kidnapping),>0,2.098361,1.000000,4.789194
30,원본,1,Hostage Taking (Kidnapping),0,0.000000,0.000000,NaN
31,원본,1,Hostage Taking (Kidnapping),>0,6.954246,2.000000,27.866912
48,평균 대체,0,Hostage Taking (Kidnapping),0,0.000000,0.000000,NaN
49,평균 대체,0,Hostage Taking (Kidnapping),>0,1.938423,1.000000,4.894725
66,평균 대체,1,Hostage Taking (Kidnapping),0,0.000000,0.000000,NaN
67,평균 대체,1,Hostage Taking (Kidnapping),>0,5.554098,3.451415,35.921635
84,중앙값 대체,0,Hostage Taking (Kidnapping),0,0.000000,0.000000,NaN
85,중앙값 대체,0,Hostage Taking (Kidnapping),>0,2.098361,1.000000,4.789194



📌 공격 유형: Unarmed Assault


,대체방법,success,attacktype1_txt,nkill_group,평균,중앙값,왜도
14,원본,0,Unarmed Assault,0,0.000000,0.0,NaN
15,원본,0,Unarmed Assault,>0,1.000000,1.0,NaN
32,원본,1,Unarmed Assault,0,0.000000,0.0,NaN
33,원본,1,Unarmed Assault,>0,4.434343,1.0,9.523517
50,평균 대체,0,Unarmed Assault,0,0.000000,0.0,NaN
51,평균 대체,0,Unarmed Assault,>0,1.000000,1.0,NaN
68,평균 대체,1,Unarmed Assault,0,0.000000,0.0,NaN
69,평균 대체,1,Unarmed Assault,>0,4.245996,1.0,9.782868
86,중앙값 대체,0,Unarmed Assault,0,0.000000,0.0,NaN
87,중앙값 대체,0,Unarmed Assault,>0,1.000000,1.0,NaN



📌 공격 유형: Unknown


,대체방법,success,attacktype1_txt,nkill_group,평균,중앙값,왜도
16,원본,0,Unknown,0,0.000000,0.00000,NaN
17,원본,0,Unknown,>0,8.254818,5.00000,5.753516
34,원본,1,Unknown,0,0.000000,0.00000,NaN
35,원본,1,Unknown,>0,6.498657,3.00000,13.620204
52,평균 대체,0,Unknown,0,0.000000,0.00000,NaN
53,평균 대체,0,Unknown,>0,7.170694,4.25262,6.727866
70,평균 대체,1,Unknown,0,0.000000,0.00000,NaN
71,평균 대체,1,Unknown,>0,6.355549,4.00000,14.520629
88,중앙값 대체,0,Unknown,0,0.000000,0.00000,NaN
89,중앙값 대체,0,Unknown,>0,6.289617,3.00000,6.256686


### **해석!**
- 전체적으로 중앙값으로 대체하는 것이 원본과 왜도가 비슷하다고 판단.

## **nkill, nwound 결측치 대체**

In [ ]:
df['nkill'] = df['nkill'].fillna(
    df.groupby(['success', 'attacktype1_txt'])['nkill']
      .transform(lambda x: x[x > 0].median() if (x > 0).any() else 0)
)

df['nwound'] = df['nwound'].fillna(
    df.groupby(['success', 'attacktype1_txt'])['nwound']
      .transform(lambda x: x[x > 0].median() if (x > 0).any() else 0)
)

In [ ]:
df[columns_to_extract].isnull().sum()

,0
eventid,0
iyear,0
imonth,0
iday,0
success,0
country_txt,0
region_txt,0
provstate,0
city,427
latitude,2531


## **natlty1_txt 대체**

In [ ]:
df[df['natlty1_txt'].isnull()]

,Unnamed: 0,eventid,iyear,imonth,iday,country_txt,region_txt,provstate,latitude,longitude,...,targtype1_txt,weaptype1_txt,nkill,nkillter,nwound,nwoundte,natlty1_txt,gname,nkill_group,nwound_group
1415,42484,199005140008,1990,5,14,South Africa,Sub-Saharan Africa,Mpumalanga,-26.102892,28.698275,...,Private Citizens & Property,Incendiary,0.0,NaN,0.0,NaN,NaN,Unknown,0,0
1835,42904,199006090020,1990,6,9,India,South Asia,Punjab,31.633979,74.872264,...,Police,Firearms,1.0,NaN,0.0,NaN,NaN,Unknown,>0,0
1906,42975,199006150008,1990,6,15,East Germany (GDR),Eastern Europe,Berlin,52.501530,13.401851,...,Private Citizens & Property,Unknown,0.0,NaN,0.0,NaN,NaN,Neo-Nazi extremists,0,0
4663,45732,199103090001,1991,3,9,Philippines,Southeast Asia,Metropolitian Manila,14.596051,120.978666,...,Religious Figures/Institutions,Explosives,2.0,NaN,2.0,NaN,NaN,Pro-Iraqi Terrorists,>0,>0
4679,45748,199103100011,1991,3,10,Mozambique,Sub-Saharan Africa,Gaza,-23.022193,32.718138,...,Food or Water Supply,Firearms,25.0,NaN,19.0,NaN,NaN,Mozambique National Resistance Movement (MNR),>0,>0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172732,213801,202105260015,2021,5,26,Afghanistan,South Asia,Ghazni,33.675278,68.421667,...,Unknown,Explosives,0.0,0.0,0.0,0.0,NaN,Unknown,0,0
173045,214114,202106090002,2021,6,9,Afghanistan,South Asia,Badghis,34.855085,63.486204,...,Unknown,Explosives,2.0,0.0,0.0,0.0,NaN,Unknown,>0,0
173191,214260,202106160017,2021,6,16,Israel,Middle East & North Africa,Southern,31.296742,34.413408,...,Unknown,Incendiary,0.0,0.0,0.0,0.0,NaN,Palestinian Extremists,0,0
173512,214581,202106270046,2021,6,26,Afghanistan,South Asia,Kunduz,36.538299,68.888879,...,Unknown,Explosives,6.0,6.0,0.0,0.0,NaN,Taliban,>0,0


In [ ]:
df[df['natlty1_txt'].isnull()].groupby('targtype1_txt').size()

,0
targtype1_txt,
Airports & Aircraft,3
Business,61
Educational Institution,4
Food or Water Supply,2
Government (Diplomatic),11
Government (General),10
Journalists & Media,8
Maritime,4
Military,5


### **타겟 대상이 unknown인 경우 natlty도 unknown으로 대체**

In [ ]:
df.loc[df['targtype1_txt'] == 'Unknown', 'natlty1_txt'] = 'Unknown'

In [ ]:
df[df['natlty1_txt'].isnull()].groupby('targtype1_txt').size()

,0
targtype1_txt,
Airports & Aircraft,3
Business,61
Educational Institution,4
Food or Water Supply,2
Government (Diplomatic),11
Government (General),10
Journalists & Media,8
Maritime,4
Military,5


### **테러 발생 국가와 natlty가 동일한지 아닌지 각 경우를 파악**

In [ ]:
# 사건 발생 국가와 공격 대상 국적이 같은지 여부를 나타내는 새로운 컬럼 생성
df['nation_match'] = df['country_txt'] == df['natlty1_txt']

# 각 타겟유형별로 nation_match의 값(참/거짓) 빈도수를 확인
result = df.groupby('targtype1_txt')['nation_match'].value_counts()

display(result)

targtype1_txt                   nation_match
Abortion Related                True              162
                                False               1
Airports & Aircraft             True              614
                                False             126
Business                        True            13217
                                False            1894
Educational Institution         True             3845
                                False             122
Food or Water Supply            True              231
                                False              10
Government (Diplomatic)         False            2097
                                True              223
Government (General)            True            18638
                                False             461
Journalists & Media             True             2131
                                False             248
Maritime                        True              190
                                False             105
Military                        True            25053
                                False            3705
NGO                             False             575
                                True              460
Other                           True              145
                                False              47
Police                          True            24145
                                False             747
Private Citizens & Property     True            43817
                                False            2972
Religious Figures/Institutions  True             4175
                                False             448
Telecommunication               True              979
                                False              17
Terrorists/Non-State Militia    True             3399
                                False             129
Tourists                        False             241
                                True              123
Transportation                  True             5035
                                False             239
Unknown                         False            7043
Utilities                       True             3869
                                False              79
Violent Political Party         True             1794
                                False              46
Name: count, dtype: int64

In [ ]:
result_pct = df.groupby('targtype1_txt')['nation_match'].value_counts(normalize=True).mul(100).round(2)
display(result_pct)

targtype1_txt                   nation_match
Abortion Related                True             99.39
                                False             0.61
Airports & Aircraft             True             82.97
                                False            17.03
Business                        True             87.47
                                False            12.53
Educational Institution         True             96.92
                                False             3.08
Food or Water Supply            True             95.85
                                False             4.15
Government (Diplomatic)         False            90.39
                                True              9.61
Government (General)            True             97.59
                                False             2.41
Journalists & Media             True             89.58
                                False            10.42
Maritime                        True             64.41
                                False            35.59
Military                        True             87.12
                                False            12.88
NGO                             False            55.56
                                True             44.44
Other                           True             75.52
                                False            24.48
Police                          True             97.00
                                False             3.00
Private Citizens & Property     True             93.65
                                False             6.35
Religious Figures/Institutions  True             90.31
                                False             9.69
Telecommunication               True             98.29
                                False             1.71
Terrorists/Non-State Militia    True             96.34
                                False             3.66
Tourists                        False            66.21
                                True             33.79
Transportation                  True             95.47
                                False             4.53
Unknown                         False           100.00
Utilities                       True             98.00
                                False             2.00
Violent Political Party         True             97.50
                                False             2.50
Name: proportion, dtype: float64

### country = natlty 경우가 97% 이상인 타겟 유형만 natlty의 결측치를 country로 대체

In [ ]:
# 1. 그룹별로 nation_match True 비율 계산 (퍼센트 단위)
result_pct = df.groupby('targtype1_txt')['nation_match'].value_counts(normalize=True).mul(100).round(2)

# 2. True 비율만 추출 (MultiIndex의 nation_match 레벨에서 True 값 선택)
true_pct = result_pct.xs(True, level='nation_match')

# 3. True 비율이 97% 이상인 범주의 리스트 생성
categories_high = true_pct[true_pct >= 97].index.tolist()
print("True 비율이 97% 이상인 범주:", categories_high)

# 4. 결측치(또는 NaN)인 'country' 컬럼만 업데이트:
mask_update = df['targtype1_txt'].isin(categories_high) & df['country_txt'].isna()
df.loc[mask_update, 'country_txt'] = 'natlty1_txt'

True 비율이 97% 이상인 범주: ['Abortion Related', 'Government (General)', 'Police', 'Telecommunication', 'Utilities', 'Violent Political Party']


In [ ]:
df[columns_to_extract].isnull().sum()

,0
eventid,0
iyear,0
imonth,0
iday,0
success,0
country_txt,0
region_txt,0
provstate,0
city,427
latitude,2531


In [ ]:
# 'coutry' 컬럼의 결측치를 "unknown"으로 채움
df['natlty1_txt'] = df['natlty1_txt'].fillna('Unknown')

In [ ]:
df[columns_to_extract].isnull().sum()

,0
eventid,0
iyear,0
imonth,0
iday,0
success,0
country_txt,0
region_txt,0
provstate,0
city,427
latitude,2531


## prov

In [ ]:
# provstate와 city의 값을 소문자로 변환한 후 동일한지 비교하여 필터링
same_values = df[df['provstate'].str.lower() == df['city'].str.lower()]

print("provstate와 city가 동일한 경우 건수:", len(same_values))
display(same_values[columns_to_extract].head(25))

provstate와 city가 동일한 경우 건수: 29417


,eventid,iyear,imonth,iday,success,country_txt,region_txt,provstate,city,latitude,longitude,attacktype1_txt,targtype1_txt,natlty1_txt,weaptype1_txt,nkill,nwound,gname
0,199001000001,1990,1,0,1,Lebanon,Middle East & North Africa,Beirut,Beirut,33.888523,35.503513,Hostage Taking (Kidnapping),Journalists & Media,Lebanon,Firearms,0.0,0.0,Unknown
4,199001010004,1990,1,1,1,Bolivia,South America,Cochabamba,Cochabamba,-17.382789,-66.166439,Bombing/Explosion,Religious Figures/Institutions,United States,Explosives,0.0,0.0,Alejo Calatayu
6,199001010006,1990,1,1,0,Israel,Middle East & North Africa,Tel Aviv,Tel Aviv,32.082970,34.811886,Assassination,Government (General),Israel,Explosives,0.0,0.0,Unknown
7,199001010007,1990,1,1,1,Bolivia,South America,Santa Cruz,Santa Cruz,-17.783219,-63.182042,Bombing/Explosion,Business,United States,Explosives,0.0,0.0,Unknown
22,199001030006,1990,1,3,0,Israel,Middle East & North Africa,Jerusalem,Jerusalem,31.771599,35.203400,Assassination,Private Citizens & Property,Israel,Explosives,0.0,0.0,Unknown
47,199001060011,1990,1,6,1,Colombia,South America,Bogota,Bogota,4.667128,-74.106056,Bombing/Explosion,Government (General),Colombia,Explosives,0.0,2.0,The Extraditables
49,199001060013,1990,1,6,1,Colombia,South America,Bogota,Bogota,4.667128,-74.106056,Bombing/Explosion,Business,Colombia,Explosives,0.0,3.0,Unknown
53,199001070003,1990,1,7,1,Japan,East Asia,Nara,nara,34.685087,135.805000,Facility/Infrastructure Attack,Government (General),Japan,Incendiary,0.0,0.0,Nara Red Army
66,199001080006,1990,1,8,0,Japan,East Asia,Kyoto,Kyoto,35.011636,135.768029,Bombing/Explosion,Government (General),Japan,Explosives,0.0,0.0,Chukakuha (Middle Core Faction)
67,199001080007,1990,1,8,1,Japan,East Asia,Tokyo,Tokyo,35.689125,139.747742,Bombing/Explosion,Government (General),Japan,Explosives,0.0,0.0,Chukakuha (Middle Core Faction)


In [ ]:
# provstate, city 컬럼의 값을 모두 소문자로 변환하여 비교
provstate_unknown = df[df['provstate'].str.lower() == 'unknown']
city_unknown = df[df['city'].str.lower() == 'unknown']
both_unknown = df[(df['provstate'].str.lower() == 'unknown') & (df['city'].str.lower() == 'unknown')]

print("provstate가 Unknown인 경우의 건수:", len(provstate_unknown))
print("city가 Unknown인 경우의 건수:", len(city_unknown))
print("provstate와 city 모두 Unknown인 경우의 건수:", len(both_unknown))


provstate가 Unknown인 경우의 건수: 2498
city가 Unknown인 경우의 건수: 8843
provstate와 city 모두 Unknown인 경우의 건수: 1640


In [ ]:
# provstate가 "Unknown"이면서, city는 "Unknown"이 아닌 경우 필터링
provstate_unknown_city_not = df[(df['provstate'].str.lower() == 'unknown') & (df['city'].str.lower() != 'unknown')]

# 데이터 출력
print("provstate가 Unknown이고, city는 Unknown이 아닌 경우 건수:", len(provstate_unknown_city_not))
display(provstate_unknown_city_not[columns_to_extract].head())

provstate가 Unknown이고, city는 Unknown이 아닌 경우 건수: 858


,eventid,iyear,imonth,iday,success,country_txt,region_txt,provstate,city,latitude,longitude,attacktype1_txt,targtype1_txt,natlty1_txt,weaptype1_txt,nkill,nwound,gname
9,199001020002,1990,1,2,1,Peru,South America,Unknown,Amnaija,NaN,NaN,Armed Assault,Police,Peru,Firearms,0.0,2.0,Shining Path (SL)
13,199001020006,1990,1,2,1,Peru,South America,Unknown,Amnaija,NaN,NaN,Armed Assault,Government (General),Peru,Incendiary,0.0,0.0,Shining Path (SL)
14,199001020007,1990,1,2,1,Peru,South America,Unknown,Amnaija,NaN,NaN,Facility/Infrastructure Attack,Government (General),Peru,Incendiary,0.0,0.0,Shining Path (SL)
36,199001050008,1990,1,5,1,Colombia,South America,Unknown,La Cuchilla,NaN,NaN,Bombing/Explosion,Utilities,Colombia,Explosives,0.0,0.0,National Liberation Army of Colombia (ELN)
37,199001060001,1990,1,6,1,India,South Asia,Unknown,Nahajia,NaN,NaN,Bombing/Explosion,Private Citizens & Property,India,Explosives,0.0,0.0,Unknown


In [ ]:
# provstate가 "unknown"이면서, city는 "unknown"이 아닌 행을 선택하는 마스크 생성
mask = (df['provstate'].str.lower() == 'unknown') & (df['city'].str.lower() != 'unknown')

# 해당 조건을 만족하는 행의 provstate 값을 city 값으로 대체
df.loc[mask, 'provstate'] = df.loc[mask, 'city']

# 결과 확인: 해당 조건을 만족하는 행들 미리보기
print("provstate 값이 city 값으로 대체된 결과:")
print(df.loc[mask, ['provstate', 'city']].head())


provstate 값이 city 값으로 대체된 결과:
      provstate         city
9       Amnaija      Amnaija
13      Amnaija      Amnaija
14      Amnaija      Amnaija
36  La Cuchilla  La Cuchilla
37      Nahajia      Nahajia


In [ ]:
# provstate, city 컬럼의 값을 모두 소문자로 변환하여 비교
provstate_unknown = df[df['provstate'].str.lower() == 'unknown']
city_unknown = df[df['city'].str.lower() == 'unknown']
both_unknown = df[(df['provstate'].str.lower() == 'unknown') & (df['city'].str.lower() == 'unknown')]

print("provstate가 Unknown인 경우의 건수:", len(provstate_unknown))
print("city가 Unknown인 경우의 건수:", len(city_unknown))
print("provstate와 city 모두 Unknown인 경우의 건수:", len(both_unknown))

provstate가 Unknown인 경우의 건수: 1640
city가 Unknown인 경우의 건수: 8843
provstate와 city 모두 Unknown인 경우의 건수: 1640


In [ ]:
# city 컬럼의 결측치(NaN)를 "unknown"으로 대체
df['city'] = df['city'].fillna('Unknown')
print(df['city'].head())

0        Beirut
1      Srinagar
2      Srinagar
3      Srinagar
4    Cochabamba
Name: city, dtype: object


In [ ]:
# provstate, city 컬럼의 값을 모두 소문자로 변환하여 비교
provstate_unknown = df[df['provstate'].str.lower() == 'unknown']
city_unknown = df[df['city'].str.lower() == 'unknown']
both_unknown = df[(df['provstate'].str.lower() == 'unknown') & (df['city'].str.lower() == 'unknown')]

print("provstate가 Unknown인 경우의 건수:", len(provstate_unknown))
print("city가 Unknown인 경우의 건수:", len(city_unknown))
print("provstate와 city 모두 Unknown인 경우의 건수:", len(both_unknown))

provstate가 Unknown인 경우의 건수: 1640
city가 Unknown인 경우의 건수: 9270
provstate와 city 모두 Unknown인 경우의 건수: 1640


In [ ]:
# provstate 또는 city가 "unknown"인 행 선택 (소문자 비교)
mask_unknown = (df['provstate'].str.lower() == 'unknown') | (df['city'].str.lower() == 'unknown')

# 위경도 데이터가 존재하는 행 선택 (NaN이 아닌 경우)
mask_latlon = df['latitude'].notna() & df['longitude'].notna()

# 두 조건을 모두 만족하는 행 필터링
df_unknown_with_latlon = df[mask_unknown & mask_latlon]

print("위경도 데이터가 있는, provstate 또는 city가 'unknown'인 경우의 건수:", len(df_unknown_with_latlon))
print(df_unknown_with_latlon.head())


위경도 데이터가 있는, provstate 또는 city가 'unknown'인 경우의 건수: 7623
     Unnamed: 0       eventid  iyear  imonth  iday country_txt     region_txt  \
8         41077  199001020001   1990       1     2    Colombia  South America   
18        41087  199001030002   1990       1     3    Colombia  South America   
25        41094  199001040001   1990       1     4        Peru  South America   
104       41173  199001110006   1990       1    11        Peru  South America   
126       41195  199001140007   1990       1    14   Sri Lanka     South Asia   

           provstate   latitude  longitude  ... weaptype1_txt  nkill nkillter  \
8       Cundinamarca   5.026003 -74.030012  ...      Firearms    0.0      NaN   
18   Valle del Cauca   3.800889 -76.641271  ...      Firearms    3.0      NaN   
25        San Martin  -7.244488 -76.825965  ...      Firearms    3.0      NaN   
104            Junin -11.161067 -75.993090  ...    Explosives    0.0      NaN   
126          Central   7.256500  80.721442  ...     